In [ ]:
import pandas as pd
import os
from multiprocessing import Pool

In [ ]:
# Read the medications CSV file and locate the problematic line
medications_file = "medications.csv"
line_number = 92156

with open(medications_file, "r") as file:
    for i, line in enumerate(file, start=1):
        if i == line_number:
            print(f"Problematic line {line_number}: {line}")
            break


In [ ]:
# Define a function to merge datasets in smaller chunks
def merge_datasets(df1, df2, on_column, how, chunk_size):
    result = pd.DataFrame()
    for chunk_df2 in df2:
        if isinstance(chunk_df2, pd.DataFrame):
            merged_chunk = pd.merge(df1, chunk_df2, on=on_column, how=how)
            result = pd.concat([result, merged_chunk], ignore_index=True)
        else:
            print("Skipping invalid chunk:", chunk_df2)
    return result



In [ ]:
# Read the CSV files in smaller chunks
diagnosis_chunks = pd.read_csv("diagnosis.csv", chunksize=1000)
operations_chunks = pd.read_csv("operations.csv", chunksize=1000)


In [ ]:
# Merge diagnosis and operations on subject_id
merged_df = pd.DataFrame()
for diagnosis_chunk, operations_chunk in zip(diagnosis_chunks, operations_chunks):
    merged_chunk = pd.merge(diagnosis_chunk, operations_chunk, on="subject_id", how="outer")
    merged_df = pd.concat([merged_df, merged_chunk], ignore_index=True)


In [ ]:
# Read the medications CSV file in chunks
medications_chunks = pd.read_csv("medications.csv", chunksize=1000)


In [ ]:
# Initialize an empty DataFrame to store the merged data
merged_df_final = pd.DataFrame()


In [ ]:
# Define a function to merge medications chunks with the existing merged data
def merge_medications_chunk(chunk_df):
    try:
        # Check if the DataFrame contains expected number of columns
        if len(chunk_df.columns) == 4:
            merged_chunk = merge_datasets(merged_df, [chunk_df], "subject_id", "outer", 1000)
            return merged_chunk
        else:
            print("Skipping chunk due to unexpected number of fields.")
            return pd.DataFrame()  # Return an empty DataFrame if unexpected number of fields
    except Exception as e:
        print(f"Error processing medications chunk: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error


In [ ]:
# Merge medications chunks using multiprocessing
with Pool() as pool:
    for i, medications_chunk in enumerate(medications_chunks):
        print(f"Processing medications chunk {i}")
        merged_chunks = pool.map(merge_medications_chunk, [medications_chunk])


Processing medications chunk 0
Processing medications chunk 1
Processing medications chunk 2
Processing medications chunk 3
Processing medications chunk 4
Processing medications chunk 5
Processing medications chunk 6
Processing medications chunk 7
Processing medications chunk 8
Processing medications chunk 9
Processing medications chunk 10
Processing medications chunk 11
Processing medications chunk 12
Processing medications chunk 13
Processing medications chunk 14
Processing medications chunk 15
Processing medications chunk 16
Processing medications chunk 17
Processing medications chunk 18
Processing medications chunk 19
Processing medications chunk 20
Processing medications chunk 21
Processing medications chunk 22
Processing medications chunk 23
Processing medications chunk 24
Processing medications chunk 25
Processing medications chunk 26
Processing medications chunk 27
Processing medications chunk 28
Processing medications chunk 29
Processing medications chunk 30
Processing medicat

ParserError: Error tokenizing data. C error: Expected 4 fields in line 92156, saw 6


In [ ]:
# Concatenate merged chunks into a single DataFrame
for i, chunk in enumerate(merged_chunks):
    print(f"Merging medications chunk {i+1}")
    merged_df_final = pd.concat([merged_df_final, chunk], ignore_index=True)

In [ ]:
# Select desired columns
selected_columns = ["subject_id", "chart_time_x", "icd10_cm", "op_id", "hadm_id", "case_id", "opdate", "sex", "weight", "height", "race", "asa", "emop", "department", "antype", "ics10_pcs", "orin_time", "orout_time", "opstart_time", "opend_time", "admission_time", "discharge_time", "anstart_time", "cpbon_time", "cpboff_time", "icuin_time", "icuout_time", "inhosp_death_time", "drug_name", "route"]
merged_df_final = merged_df_final[selected_columns]

# Remove duplicate rows based on subject_id
merged_df_final = merged_df_final.drop_duplicates(subset="subject_id")

# Write to a new CSV file
merged_df_final.to_csv("merged_data.csv", index=False)
